In [61]:
todo : 现在缺格式与安全保护问题，理论上已经能跑了但是还没跑过。

#main.py

import rsa
import numpy as np
import pandas as pd
import random
import math
import os

namelist = ['同学0','同学1','同学2','同学3','同学4','同学5','同学6','同学7']
domitory = [4,4]

namedict = {v:i for i,v in enumerate(namelist)}
n = len(namelist)
dom_num = len(domitory)
assert(n == dom_num*4)




publicKey, privateKey = rsa.newkeys(512)

with open('public.pem', 'wb') as f:
    f.write(publicKey.save_pkcs1())






# tagnum = 7




class info:
    # def __init__(self, wantnum, numbiasweight, peoplebiasweight, peoplebias):
    def __init__(self,peoplebias):
        self.peoplebias = peoplebias
        # self.wantnum = wantnum
        # self.numbiasweight = numbiasweight
        # self.peoplebiasweight = peoplebiasweight

allinfo = [0 for _ in range(n)]

#无验证提取

def get_info(df):

    nameid = namedict[df.iloc[0,1]]

    # numbiasweight = df.iloc[2,1]
    # wantnum = df.iloc[1,1]
    # peoplebiasweight = df.iloc[3,1]

    peoplebias = df.iloc[0:n,3].values
    
    # allinfo[nameid] = info(wantnum, numbiasweight, peoplebiasweight, peoplebias)
    allinfo[nameid] = info(peoplebias)

bias = np.zeros((n,n))

# dom_bias = np.zeros(n)

def build_bias_from_allinfo():
    for i in range(n):
        for j in range(n):
            if i != j:
                bias[i,j] += allinfo[i].peoplebias[j] + allinfo[j].peoplebias[i]
        # dom_bias[i] += allinfo[i].wantnum


#check when decrypt
def decrypt2pandaframe(filename):
    with open(filename, 'rb') as f:
        text = f.read()
        res = []
        for i in range(0, len(text), 64):
            res.append(rsa.decrypt(text[i:i + 64],privateKey))
        byte_data = b''.join(res)
        df = pd.read_excel(byte_data)
        return df


print("收集完毕后，请输入'y'继续")
while(True):
    if input() == 'y':
        files = filter(lambda x: x.endswith('.pem'), os.listdir('.'))
        if len(list(files)) != n:
            print("请收集完毕后，再继续")
            continue
        try:
            dfs = [decrypt2pandaframe(f) for f in files]
        except:
            print("解密失败，请重新收集")#todo要报更详细的解密失败信息
            continue
        try:
            infos = [get_info(df) for df in dfs]
        except:
            print("格式失败，请重新收集")#todo要报更详细的格式失败信息
            continue
        build_bias_from_allinfo()
        break
    else:
        print("请输入'y'继续")

#decode

#dfs check



# format check

# range check

# problem simplify




#同学的分配标号
#有床位但没alloc用-1代替。
cnt = -1
dom_alloc =[[cnt := cnt + 1 for j in range (domitory[i])] for i in range (dom_num)]
#每个寝室的满意分值
dom_score = [0 for _ in range(dom_num)]

#对单个room计算分数
def calc_score(room):
    valid_room = [i for i in room if i != -1]
    stu_num = len(valid_room)
    if stu_num <= 1:
        return 0
    ans = sum([bias[i][j] for i in valid_room for j in valid_room if i < j])
    return ans

for i in range (dom_num):
    dom_score[i] = calc_score(dom_alloc[i])

# 这里还没有使用dom_bias，因为不知道会不会有寝室人数不同的情况。再说吧。
T = 1000
RETIME = 10000
renew = RETIME
while renew:
    #降温
    T *= 0.999
    #随机选择两个寝室
    x,y = random.sample(range(dom_num),2)
    #ramdom_swap 两个寝室的两人
    roomx = dom_alloc[x].copy()
    roomy = dom_alloc[y].copy()
    xi = random.randrange(domitory[x])
    yi = random.randrange(domitory[y])
    roomx[xi], roomy[yi] = roomy[yi], roomx[xi]
    scorex = calc_score(roomx)
    scorey = calc_score(roomy)
    score_up = scorex + scorey - dom_score[x] - dom_score[y]
    #如果分数升高了，则进行交换，不然以一定概率交换
    if score_up >= 0 or random.random() < math.exp(score_up/T):
        dom_score[x] = scorex
        dom_score[y] = scorey
        dom_alloc[x] = roomx
        dom_alloc[y] = roomy
        renew = RETIME
    else:
        renew -= 1
    # print(sum(dom_score))
    


# print(sum(dom_score))
# print(f'{dom_score=},{dom_alloc=}')
# print(bias)

alloc_with_names = [[namelist[i] for i in dom_alloc[j]] for j in range (dom_num)]

print(alloc_with_names)


In [62]:
#客户端
#encrypt.py

#可以打包成环境无关的可执行文件？以防库的问题。

import rsa
with open('宿舍分配填写表.xlsx', 'rb') as f:
    text = f.read()
    res = []
    for i in range(0, len(text), 53):
        res.append(rsa.encrypt(text[i:i + 53],publicKey))
    byte_data = b''.join(res)
    with open('encrypted.bin', 'wb') as ff:
        ff.write(byte_data)

In [ ]:
#客户端
#check_format_and_range.py
import pandas as pd
